**스타트캠프 교육 자료 저작권 및 사용 제한 안내**
- `스타트캠프 교육 자료의 모든 권리는 저작권자에게 있습니다.`
- `이 자료는 동의없이 외부로 유출하거나 개인적인 용도로 영리적인 목적을 위해 사용할 수 없습니다.`

## GPT 3.5버전 API 실습 [Tool 만들기]
### Tool 기능이란?
+ GPT 모델의 기본 언어 생성 기능 외에 특정 작업을 수행하기 위해 특별히 설계된 도구를 사용할 수 있도록 해주는 확장 기능. ex) 이미지 생성, 인터넷 검색, 파이썬 코드 실행 등
+ GPT 모델은 특정 작업에 대해 더 정확하고 유용한 결과를 제공

### 0. 환경변수 설정

In [ ]:
# 필요 라이브러리 설치
!pip install openai

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

#API-KEY 설정
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
#모델 설정
GPT_MODEL = 'gpt-3.5-turbo'

### 1. 실시간 기상 정보를 알려주는 chatbot 만들기
+ 날씨 호출 tool 만들기

In [3]:
import json
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

#GPT API호출 및 응답 가져오기
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [4]:
#Tool 선언
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",                #함수 이름
            "description": "현재 날씨 정보를 가져옵니다", #함수 설명
            "parameters": {                               #함수 파라미터 지정
                "type": "object",
                "properties": {
                    "location": {                         #파라미터 명
                        "type": "string",                 #파라미터 타입
                        "description": "도시 또는 지역을 입력하세요, 예: 서울, 충청도", #파라미터 설명 및 예시
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "사용할 온도 단위를 입력하세요. 사용자의 위치에 따라 추론됩니다. 예를 들어 대한민국의 도시들은 섭씨 온도입니다.",
                    },
                },
                "required": ["location", "unit"],         #필수 파라미터 지정
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "현재 날짜부터 N일 간의 날씨 예보를 가져옵니다",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "도시 또는 지역을 입력하세요, 예: 서울, 충청도",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "사용할 온도 단위를 입력하세요. 사용자의 위치에 따라 추론됩니다. 예를 들어 대한민국의 도시들은 섭씨 온도입니다",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "예보할 일수를 입력하세요",
                    }
                },
                "required": ["location", "unit", "num_days"]
            },
        }
    },
]

#함수 구현 (원래는 날씨 API호출 해야하지만 편의상 랜덤하게 지정)
import random

def get_current_weather(location, unit):

    weather_conditions = ["맑음", "흐림", "비", "눈", "안개", "폭풍"]
    weather = random.choice(weather_conditions)
    temperature = round(random.uniform(-10, 35), 1) if unit == "celsius" else round(random.uniform(14, 95), 1)

    return str({"날씨": weather, "기온": temperature, "단위": unit})

def get_n_day_weather_forecast(location, unit, num_days):

    weather_conditions = ["맑음", "흐림", "비", "눈", "안개", "폭풍"]
    forecasts = []

    for day in range(1, num_days+1):
        weather = random.choice(weather_conditions)
        temperature = round(random.uniform(-10, 35), 1) if unit == "celsius" else round(random.uniform(14, 95), 1)
        forecasts.append({"일차": day, "날씨": weather, "기온": temperature, "단위": unit})

    return str(forecasts)

# 예시 호출
print(get_current_weather("대전", "fahrenheit"))
print(get_n_day_weather_forecast("서울", "celsius", 5))


{'날씨': '눈', '기온': 80.8, '단위': 'fahrenheit'}
[{'일차': 1, '날씨': '폭풍', '기온': 22.0, '단위': 'celsius'}, {'일차': 2, '날씨': '눈', '기온': 5.6, '단위': 'celsius'}, {'일차': 3, '날씨': '비', '기온': 4.9, '단위': 'celsius'}, {'일차': 4, '날씨': '눈', '기온': -0.2, '단위': 'celsius'}, {'일차': 5, '날씨': '눈', '기온': 12.3, '단위': 'celsius'}]


+ 프롬프트에 따라 어떤 Tool이 호출되는지 확인

In [9]:
messages = []

messages.append({"role": "system", "content": "당신은 기상 정보를 알려주는 챗봇입니다."})
messages.append({"role": "system", "content": "사용할 함수의 값에 대해 추측하지 마세요. 사용자의 요청이 모호한 경우 명확히 물어보세요."})
messages.append({"role": "user", "content": "오늘 날씨 어때?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
print(assistant_message)


ChatCompletionMessage(content='어떤 도시의 날씨 정보를 원하시나요? 예를 들어, "서울"이나 "충청도"와 같이 구체적인 지역을 말씀해주세요.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [10]:
messages = []
messages.append({"role": "system", "content": "사용할 함수의 값에 대해 추측하지 마세요. 사용자의 요청이 모호한 경우 명확히 물어보세요."})
messages.append({"role": "user", "content": "현재 서울의 날씨를 알고싶어."})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
print(assistant_message)


ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_MQoR32GoOX6uSflPIOmZG95r', function=Function(arguments='{"location":"서울","unit":"celsius"}', name='get_current_weather'), type='function')])


In [11]:
messages = []
messages.append({"role": "system", "content": "사용할 함수의 값에 대해 추측하지 마세요. 사용자의 요청이 모호한 경우 명확히 물어보세요."})
messages.append({"role": "user", "content": "대전 5일간 날씨는 어때?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
print(assistant_message)


ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_y1IXQc47GFmqniGD5e11l759', function=Function(arguments='{"location":"대전","unit":"celsius","num_days":5}', name='get_n_day_weather_forecast'), type='function')])


+ 날씨 정보를 제공하는 챗봇

In [12]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

messages = [
    {"role": "system", 
     "content":
        """당신은 날씨 정보를 제공하는 챗봇입니다.
        
        준수해야 할 규칙:
        1. 위치가 불명확할 경우 반드시 사용자에게 확인하세요
        2. 온도 단위가 입력되지 않은 경우 단위는 입력된 지역에 따라 추론하세요.
        3. 날씨 정보 제공시 온도와 날씨 상태를 자연스러운 문장으로 설명하세요
        4. 사용자의 질문이 날씨와 관계없는 경우 정중히 날씨 관련 질문만 답변 가능함을 안내하세요
        5. 함수 호출이 아닌 답변의 경우 마크다운 형식으로 정리하세요
        
        예시 응답:
        - "서울의 현재 기온은 20도이며, 날씨는 맑습니다."
        - "서울의 앞으로 3일간 날씨 예보는 다음과 같습니다:
            | 일자  | 날씨   | 기온 (°C) |
            |-------|-------|-----------|
            | 1일   | 눈     | 25.2      |
            | 2일   | 안개   | -7.4      |
            | 3일   | 맑음   | -1.8      |"
        - "죄송합니다. 위치를 말씀해 주시겠어요? 어느 지역의 날씨를 알고 싶으신가요?"
        """
    }
]


# '종료' 입력 전까지 대화
user_input = ''
while True:
    user_input = input("You: ")
    if "종료" in user_input : break
    messages.append({"role": "user", "content": user_input})

    chat_response = chat_completion_request(messages, tools)

    # 1단계: 함수 호출로 이어질 수 있는 내용을 포함한 프롬프트
    # 이 경우, 모델은 사용자가 요청한 질문이 설정한 도구 설명에 부합되는지 확인 후 해당 함수 호출.
    assistant_message = chat_response.choices[0].message
    messages.append(assistant_message)

    # 2단계: 모델의 응답에 도구 호출이 포함되어 있는지 확인합니다.
    if assistant_message.tool_calls :
        for msg in assistant_message.tool_calls :
          # 3단계: 함수를 호출하고 결과를 얻습니다. 결과를 messages 리스트에 추가합니다.
          if "function" in msg.type:
              tool_id = msg.id  #툴 ID
              function_call = msg.function #함수 정보 가져오기
              function_name = function_call.name  #함수 이름
              function_args = json.loads(function_call.arguments) #함수 파라미터

              # 실제 함수 호출
              if function_name == "get_n_day_weather_forecast":
                  result = get_n_day_weather_forecast(**function_args)
                  # print("get_n_day_weather_forecast 호출했습니다.", result)

              elif function_name == "get_current_weather":
                  result = get_current_weather(**function_args)
                  # print("get_current_weather 호출했습니다.", result)
              messages.append({
                  "role": "tool",
                  "tool_call_id": tool_id,
                  "name": function_name,
                  "content": result
              })
        # 4단계: 함수 응답이 추가된 messages 리스트로 GPT API를 호출합니다.
        # tool_calls이 호출되면 반드시 해당 id에 대한 응답이 필요합니다.
        chat_response = chat_completion_request(messages)

    #결과 출력
    assistant_reply = chat_response.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_reply})
    print(f"Assistant: {assistant_reply}")

Assistant: **청주의 현재 기온은 -2.5도이며, 날씨는 안개가 끼었습니다.**
Assistant: **청주의 앞으로 5일간 날씨 예보는 다음과 같습니다:**
| 일자  | 날씨   | 기온 (°C) |
|-------|-------|-----------|
| 1일   | 눈     | 17.4      |
| 2일   | 비     | 21.6      |
| 3일   | 흐림   | 11.3      |
| 4일   | 폭풍   | 5.8       |
| 5일   | 눈     | -1.7      |
Assistant: **죄송합니다. 위치를 말씀해 주시겠어요? 어느 지역의 날씨를 알고 싶으신가요?**
Assistant: **서울의 내일 날씨는 눈이며, 기온은 33.3도입니다.**


---